<a href="https://colab.research.google.com/github/vzav/exemples/blob/master/kaggle_comp(_ASHRAE_Great_Energy_Predictor_III)_22_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
#from datetime import datetime
from datetime import date

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"zavvit","key":"46da327c6d1bdc8d5ec4928f12a5caa5"}'}

In [0]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 62 Oct 22 12:19 kaggle.json


In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/ 
!chmod 600 ~/.kaggle/kaggle.json 

In [0]:
!kaggle competitions download -c ashrae-energy-prediction

 96% 85.0M/88.4M [00:00<00:00, 120MB/s] 
100% 88.4M/88.4M [00:00<00:00, 121MB/s]
  0% 0.00/44.5k [00:00<?, ?B/s]
100% 44.5k/44.5k [00:00<00:00, 47.5MB/s]
 94% 156M/167M [00:01<00:00, 90.1MB/s]
100% 167M/167M [00:01<00:00, 102MB/s] 
  0% 0.00/2.53M [00:00<?, ?B/s]
100% 2.53M/2.53M [00:00<00:00, 140MB/s]
  0% 0.00/1.27M [00:00<?, ?B/s]
100% 1.27M/1.27M [00:00<00:00, 80.2MB/s]
 88% 105M/120M [00:01<00:00, 85.3MB/s] 
100% 120M/120M [00:01<00:00, 107MB/s] 


In [0]:
!unzip \*.zip

Archive:  weather_test.csv.zip
  inflating: weather_test.csv        

Archive:  test.csv.zip
  inflating: test.csv                

Archive:  weather_train.csv.zip
  inflating: weather_train.csv       

Archive:  train.csv.zip
  inflating: train.csv               

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   

5 archives were successfully processed.


In [0]:
building_metadata = pd.read_csv('building_metadata.csv')
building_metadata.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


In [0]:
train = pd.read_csv('train.csv')
train.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


In [0]:
y = train['meter_reading']
y.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: meter_reading, dtype: float64

In [0]:
train = train.drop('meter_reading', axis = 1)
train.head()

,building_id,meter,timestamp
0,0,0,2016-01-01 00:00:00
1,1,0,2016-01-01 00:00:00
2,2,0,2016-01-01 00:00:00
3,3,0,2016-01-01 00:00:00
4,4,0,2016-01-01 00:00:00


In [0]:
weather_train = pd.read_csv('weather_train.csv')
weather_train.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [0]:
test = pd.read_csv('test.csv')
print(test.columns)
test = test.drop('row_id', axis = 1)
test.head()

Index(['row_id', 'building_id', 'meter', 'timestamp'], dtype='object')


,building_id,meter,timestamp
0,0,0,2017-01-01 00:00:00
1,1,0,2017-01-01 00:00:00
2,2,0,2017-01-01 00:00:00
3,3,0,2017-01-01 00:00:00
4,4,0,2017-01-01 00:00:00


In [0]:
weather_test = pd.read_csv('weather_test.csv')
weather_test.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2017-01-01 00:00:00,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
1,0,2017-01-01 01:00:00,17.8,2.0,12.8,0.0,1022.0,130.0,3.1
2,0,2017-01-01 02:00:00,16.1,0.0,12.8,0.0,1021.9,140.0,3.1
3,0,2017-01-01 03:00:00,17.2,0.0,13.3,0.0,1022.2,140.0,3.1
4,0,2017-01-01 04:00:00,16.7,2.0,13.3,0.0,1022.3,130.0,2.6


In [0]:
print ('building_metadata shape: ', building_metadata.shape)
print ('Train shape: ', train.shape)
print ('weather_train shape: ', weather_train.shape)
print ('Test shape: ', test.shape)
print ('weather_test shape: ', weather_test.shape)

building_metadata shape:  (1449, 6)
Train shape:  (20216100, 3)
weather_train shape:  (139773, 9)
Test shape:  (41697600, 3)
weather_test shape:  (277243, 9)


Merging everything into two datasets: train and test

In [0]:
df = train.merge(building_metadata, how='left', on='building_id') # альтернатива  left_on = "building_id", right_on = "building_id"
df = df.merge(weather_train, how='left', left_on= ['site_id', 'timestamp'], right_on= ['site_id', 'timestamp'])

In [0]:
test = test.merge(building_metadata, on='building_id', how='left')
test = test.merge(weather_test, on=['site_id', 'timestamp'], how='left')
del weather_train, weather_test,building_metadata, train # ,train можно не удалаять если переименовать df в train
gc.collect(); # это что? похже на зачистку мусора

In [0]:
print ('df shape: ', df.shape)
print ('test shape: ', test.shape)

NameError: ignored

Explore data

In [0]:
df.columns.values  # если df.columns ajhvbhetncz Index([...] dtype='object), если df.columns.values - формируется array([..] dtype=object)

In [0]:
df.info()

In [0]:
#df.info()   # тут видно тип данных int64  object  float64
# df.isnull()  # так вся матрица с определением является ли поле null 
#df.display  # 'DataFrame' object has no attribute 'display'
#df.describe()  # count mean std 25% 50% 75% max
#df.value_counts() # 'DataFrame' object has no attribute 'value_counts'

df.isna().sum() # количество пропущенных значений по каждой колонке

## альтернативный но громоздкий вариант просмотра пропусков по колонкам. и работает медленнее
# for col in df.columns:
#   print(col, df[df[col].isnull()].shape[0])


In [0]:
df.describe()

In [0]:
df[(df['building_id']==1099) & (df['meter_reading'] > 3000000)] # аномалия - выкинуть?

In [0]:
# удалить строки с пропусками по    air_temperature      dew_temperature     wind_speed 
# дополнительно посмотреть кол-во по этим полям в тест - может не стоит
df = df.dropna(axis=0,subset=['air_temperature','dew_temperature','wind_speed'])   # , не сумма так как два или три в одной строке могут быть  df.shape Out: (20067705, 16)
.shape

In [0]:
# заменить другие пропуски по намеченым категориальным данным пропуски
df['year_built']= df['year_built'].fillna(date.today().year+1) # пустое значенеи меняем на 2020 год, для того чтоб при вычитании получить -1 - как категория
df['floor_count','cloud_coverage','wind_direction']= df['floor_count','cloud_coverage','wind_direction'].fillna(-1)
df.head(50)

In [0]:
# # today = date.today().isoformat()
# # today
# from datetime import datetime
# now = datetime.today().isoformat()
# print(now)

2019-10-21T14:09:37.040771


In [0]:
# from datetime import date
# todayDate = date.today()
# print(todayDate)
# print(todayDate.year)

2019-10-21
2019


In [0]:
# print(date.today().year+1)

2020


In [0]:
# категориями сделать - время(готово +)     year_built (в десятках лет)==возраст(+),     floor_count(замена на группы -1, 1, 2, 3, 4, 5-9, 10-17, 18-35, 35+),     cloud_coverage,    wind_direction
# возможно категориями сделать - precip_depth_1_hr  sea_level_pressure 

In [0]:
# категориями сделать - время
df["timestamp_new"] = pd.to_datetime(df["timestamp"])  
df["hour"] = df["timestamp_new"].dt.hour  # тут пока на веру берем что новая колонка hour соответсвует времени
df["day"] = df["timestamp_new"].dt.day
df["weekend"] = df["timestamp_new"].dt.weekday
df["month"] = df["timestamp_new"].dt.month
df.head()

In [0]:
df['year_built_range'] = round((date.today().year - df['year_built'])/10) # от текщего года отнимаем год постройки - перевести в категории (22 категории где-то). проверить

In [0]:
# X = (df["cloud_coverage"].max() - df["cloud_coverage"].min()) / 10
# df["cloud_coverage"] = round(df["cloud_coverage"] / X)

def categories_rnd(a, n):  #  - количество градаций. ( возможна ошибка так как чисел может быть больше). перделать градацию
  if a != -1:
    x = (a.max() - a.min())/n
    b = round (a / x)
  else:
    b = a
  return b

ls = [1,2,3,5,7,9]
d = catories_rnd(ls)
print(d)



In [0]:
# floor_count(замена на группы -1, 1, 2, 3, 4, 5-9, 10-17, 18-35, 35+),  
# возможно категориями сделать - precip_depth_1_hr  sea_level_pressure

In [0]:
df["cloud_coverage_range"] = categories_rnd(df["cloud_coverage"], 10)  # 10 - корличество градаций
df["wind_direction_range"] = categories_rnd(df["wind_direction"], 8)
df["precip_depth_1_hr_range"] = categories_rnd(df["precip_depth_1_hr"], 10)
df["sea_level_pressure_range"] = categories_rnd(df["sea_level_pressure"], 10)


In [0]:
# превратить в категории

In [0]:
# и не забыть все манипуляции над тест провести. просмотрет еще раз как объеденяется

In [0]:
# нормализация числовых данных

In [0]:
y.describe()

In [0]:
plt.figure()
# y.plot  # и так еще можно
#sns.distplot(y)

#sns.distplot(np.log(y))
plt.title('Distribution of meter_reading(y)')
plt.show()

In [0]:
# перед запуском модкли удалить лишние столбцы:
# delete target from df   df_1 = 
# delete timestamp_new